In [17]:
import os
import random

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

from data_generator import DataGenerator

In [31]:
labels = dict()
file_names_diffusion14 = []
for file in os.listdir(r"owoce\apple_diffusion_1_4"):
    file_names_diffusion14.append(r"owoce\apple_diffusion_1_4\\" + file)
    labels[r"owoce\apple_diffusion_1_4\\"+file] = 0

file_names_diffusion15 = []
for file in os.listdir(r"owoce\apple_diffusion_1_5"):
    file_names_diffusion15.append(r"owoce\apple_diffusion_1_5\\" + file)
    labels[r"owoce\apple_diffusion_1_5\\"+file] = 0

file_names_diffusion21 = []
for file in os.listdir(r"owoce\apple_diffusion_2_1"):
    file_names_diffusion21.append(r"owoce\apple_diffusion_2_1\\" + file)
    labels[r"owoce\apple_diffusion_2_1\\"+file] = 0

file_names_realistic14 = []
for file in os.listdir(r"owoce\apple_realistic_1_4"):
    file_names_realistic14.append(r"owoce\apple_realistic_1_4\\" + file)
    labels[r"owoce\apple_realistic_1_4\\"+file] = 0

file_names_apple_detection = []
for file in os.listdir(r"owoce\Apple Detection.v1i.coco\train"):
    file_names_apple_detection.append(r"owoce\Apple Detection.v1i.coco\train\\" + file)
    labels[r"owoce\Apple Detection.v1i.coco\train\\"+file] = 1

file_names_fruitnet = []
for file in os.listdir(r"owoce\Apple_Good"):
    file_names_fruitnet.append(r"owoce\Apple_Good\\" + file)
    labels[r"owoce\Apple_Good\\"+file] = 1

file_names_net = []
for file in os.listdir(r"owoce\net"):
    file_names_net.append(r"owoce\net\\"+file)
    labels[r"owoce\net\\"+file] = 1

file_names_dalle = []
for file in os.listdir(r"owoce\dalle"):
    file_names_dalle.append(r"owoce\dalle\\"+file)
    labels[r"owoce\dalle\\"+file] = 0

file_names_firefly = []
for file in os.listdir(r"owoce\fake photos - firefly"):
    file_names_firefly.append(r"owoce\fake photos - firefly\\"+file)
    labels[r"owoce\fake photos - firefly\\"+file] = 0

file_names_by_phone = []
for file in os.listdir(r"owoce\real photos"):
    file_names_by_phone.append(r"owoce\real photos\\" + file)
    labels[r"owoce\real photos\\"+file] = 1

file_names_real_with_custom_background = []
for file in os.listdir(r"owoce\real apples"):
    file_names_real_with_custom_background.append(r"owoce\real apples\\" + file)
    labels[r"owoce\real apples\\"+file] = 1

file_names_fake_with_custom_background = []
for file in os.listdir(r"owoce\fake apples"):
    file_names_fake_with_custom_background.append(r"owoce\fake apples\\" + file)
    labels[r"owoce\fake apples\\"+file] = 0

In [32]:
random.seed(1234)

partition = {
    "train": [],
    "validation": [],
    "test": [],
    "test_only_diffusion": [],
    "test_only_firefly": [],
    "test_only_dalle": [],
    "test_only_custom_background": []
}

random.shuffle(file_names_diffusion14)

partition["train"] += file_names_diffusion14[:(int(0.5 * len(file_names_diffusion14)))]
partition["validation"] += file_names_diffusion14[(int(0.5 * len(file_names_diffusion14))):(int(0.6 * len(file_names_diffusion14)))]
partition["test"] += file_names_diffusion14[(int(0.6 * len(file_names_diffusion14))):]
partition["test_only_diffusion"] += file_names_diffusion14[(int(0.6 * len(file_names_diffusion14))):]

random.shuffle(file_names_diffusion15)

partition["train"] += file_names_diffusion15[:(int(0.5 * len(file_names_diffusion15)))]
partition["validation"] += file_names_diffusion15[(int(0.5 * len(file_names_diffusion15))):(int(0.6 * len(file_names_diffusion15)))]
partition["test"] += file_names_diffusion15[(int(0.6 * len(file_names_diffusion15))):]
partition["test_only_diffusion"] += file_names_diffusion15[(int(0.6 * len(file_names_diffusion15))):]

random.shuffle(file_names_diffusion21)

partition["train"] += file_names_diffusion21[:(int(0.5 * len(file_names_diffusion21)))]
partition["validation"] += file_names_diffusion21[(int(0.5 * len(file_names_diffusion21))):(int(0.6 * len(file_names_diffusion21)))]
partition["test"] += file_names_diffusion21[(int(0.6 * len(file_names_diffusion21))):]
partition["test_only_diffusion"] += file_names_diffusion21[(int(0.6 * len(file_names_diffusion21))):]

random.shuffle(file_names_realistic14)

partition["train"] += file_names_realistic14[:(int(0.5 * len(file_names_realistic14)))]
partition["validation"] += file_names_realistic14[(int(0.5 * len(file_names_realistic14))):(int(0.6 * len(file_names_realistic14)))]
partition["test"] += file_names_realistic14[(int(0.6 * len(file_names_realistic14))):]
partition["test_only_diffusion"] += file_names_realistic14[(int(0.6 * len(file_names_realistic14))):]

random.shuffle(file_names_apple_detection)

partition["train"] += file_names_apple_detection[:(int(0.5 * len(file_names_apple_detection)))]
partition["validation"] += file_names_apple_detection[(int(0.5 * len(file_names_apple_detection))):(int(0.6 * len(file_names_apple_detection)))]
partition["test"] += file_names_apple_detection[(int(0.6 * len(file_names_apple_detection))):]

random.shuffle(file_names_fruitnet)

partition["train"] += file_names_fruitnet[:(int(0.5 * len(file_names_fruitnet)))]
partition["validation"] += file_names_fruitnet[(int(0.5 * len(file_names_fruitnet))):(int(0.6 * len(file_names_fruitnet)))]
partition["test"] += file_names_fruitnet[(int(0.6 * len(file_names_fruitnet))):]

random.shuffle(file_names_net)

partition["train"] += file_names_net[:(int(0.5*len(file_names_net)))]
partition["validation"] += file_names_net[(int(0.5*len(file_names_net))):(int(0.6*len(file_names_net)))]
partition["test"] += file_names_net[(int(0.6*len(file_names_net))):]

random.shuffle(file_names_dalle)

partition["train"] += file_names_dalle[:(int(0.5*len(file_names_dalle)))]
partition["validation"] += file_names_dalle[(int(0.5*len(file_names_dalle))):(int(0.6*len(file_names_dalle)))]
partition["test"] += file_names_dalle[(int(0.6*len(file_names_dalle))):]
partition["test_only_dalle"] += file_names_dalle[(int(0.6*len(file_names_dalle))):]

random.shuffle(file_names_firefly)

partition["train"] += file_names_firefly[:(int(0.5*len(file_names_firefly)))]
partition["validation"] += file_names_firefly[(int(0.5*len(file_names_firefly))):(int(0.6*len(file_names_firefly)))]
partition["test"] += file_names_firefly[(int(0.6*len(file_names_firefly))):]
partition["test_only_firefly"] = file_names_firefly[(int(0.6*len(file_names_firefly))):]

random.shuffle(file_names_by_phone)

partition["train"] += file_names_by_phone[:(int(0.5 * len(file_names_by_phone)))]
partition["validation"] += file_names_by_phone[(int(0.5 * len(file_names_by_phone))):(int(0.6 * len(file_names_by_phone)))]
partition["test"] += file_names_by_phone[(int(0.6 * len(file_names_by_phone))):]

partition["test_only_custom_background"] += file_names_real_with_custom_background
partition["test_only_custom_background"] += file_names_fake_with_custom_background

In [37]:
training_generator = DataGenerator(partition["train"], labels, batch_size=32)
validation_generator = DataGenerator(partition["validation"], labels, batch_size=32)

test_generator = DataGenerator(partition["test"], labels, batch_size=32)
test_generator_diffusion = DataGenerator(partition["test_only_diffusion"], labels, batch_size=32)
test_generator_firefly = DataGenerator(partition["test_only_firefly"], labels, batch_size=32)
test_generator_dalle = DataGenerator(partition["test_only_dalle"], labels, batch_size=24)
test_generator_only_custom_background = DataGenerator(partition["test_only_custom_background"], labels, batch_size=10)

In [ ]:
conv_base = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(192, 192, 3)
)
for layer in conv_base.layers:
    layer.trainable = False

x = conv_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dense(64, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)
model = Model(conv_base.input, predictions)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics="accuracy")

In [ ]:
history = model.fit(training_generator, epochs=10, validation_data=validation_generator)

In [21]:
model.save("resnet50_2layers.h5")

In [ ]:
# model = load_model("resnet50_2layers.h5")

In [35]:
def do_and_summarize_test(model, test_generator):
    # data preparation
    data = []
    labels = []
    max_iter = 100
    i = 0
    for d, l in test_generator:
        data.append(d)
        labels.append(l)
        i += 1
        if i == max_iter:
            break

    data = np.array(data)
    data = np.reshape(data, (data.shape[0] * data.shape[1],) + data.shape[2:])

    labels = np.array(labels)
    labels = np.reshape(labels, (labels.shape[0] * labels.shape[1],) + labels.shape[2:])
    number_of_cases = labels.shape[0]
    labels = np.reshape(labels, (number_of_cases, 1))

    # accuracy, precision, recall and auc
    score = model.evaluate(data, labels, verbose=0)
    print("Test Accuracy:", score[1])

    y_pred = model.predict(data)
    for x in range(number_of_cases):
        if y_pred[x][0] < 0.5:
            y_pred[x][0] = 0
        else:
            y_pred[x][0] = 1
    y_pred_labels = y_pred
    y_test_labels = labels.astype("float32")
    print("Classification Report:")
    print(classification_report(y_test_labels, y_pred_labels))

    try:
        auc = roc_auc_score(y_test_labels, y_pred)
        print("AUC:", auc)
    except ValueError:
        pass

    # confusion matrix
    cm = confusion_matrix(y_test_labels, y_pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.show()

In [ ]:
do_and_summarize_test(model, test_generator)

In [ ]:
do_and_summarize_test(model, test_generator_dalle)

In [ ]:
do_and_summarize_test(model, test_generator_firefly)

In [ ]:
do_and_summarize_test(model, test_generator_diffusion)

In [ ]:
do_and_summarize_test(model, test_generator_only_custom_background)